# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)


In [ ]:
import numpy as np
import pandas as pd
import operator
import nltk
from scipy import stats 
from collections import defaultdict

!pip install conllutils
import conllutils

!pip install conll-df
from conll_df import conll_df # https://github.com/interrogator/conll-df

**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.

In [ ]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

fatal: destination path 'UD_English-GUM' already exists and is not an empty directory.


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [ ]:
directory = 'UD_English-GUM'
word = 'w'
position = 'x'

train = f'{directory}/en_gum-ud-train.conllu'
test = f'{directory}/en_gum-ud-test.conllu'
dev = f'{directory}/en_gum-ud-dev.conllu'

train_df = conll_df(train, file_index=False)[[word, position]]
test_df = conll_df(test, file_index=False)[[word, position]]
dev_df = conll_df(dev, file_index=False)[[word, position]]

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [ ]:
# Helper functions
def create_word_tag_dict(data: pd.DataFrame):
  word_to_tag_dict = defaultdict(list)

  for word, tag in data.itertuples(index=False):
    word_to_tag_dict[word].append(tag)
  return {key: stats.mode(val).mode[0] for key, val in word_to_tag_dict.items()}

def get_most_frequent_tag(data: pd.DataFrame, column=position):
  return data[column].value_counts().idxmax()

In [ ]:
class simple_tagger:
  def __init__(self):
    self.dictionary = {}
    self.tag = ''

  def train(self, data):
    self.word_to_modal_tag_dict = create_word_tag_dict(data)
    self.most_frequent_tag = get_most_frequent_tag(data)
  
  def evaluate(self, data):
    correct_word_counter = 0
    correct_sentence_counter = 0
    num_words = len(data)
    
    df_grouped_by_sentence = data.groupby('s')
    num_sentences = df_grouped_by_sentence.ngroups

    for sentence_num, sentence_df in df_grouped_by_sentence:
      sentence_correct = True
      for _, (word, true_tag) in sentence_df.iterrows():
        predicted_tag = self.word_to_modal_tag_dict.get(word, self.most_frequent_tag)
        if predicted_tag == true_tag:
          correct_word_counter += 1
        else:
          sentence_correct = False
      if sentence_correct: 
        correct_sentence_counter += 1

    w = round(correct_word_counter / num_words * 100, 4)
    s = round(correct_sentence_counter / num_sentences * 100, 4)
    
    return w, s

In [ ]:
tagger = simple_tagger()
tagger.train(train_df)

word_train, sentence_train = tagger.evaluate(train_df)
word_test, sentence_test = tagger.evaluate(test_df)
word_dev, sentence_dev = tagger.evaluate(dev_df)

In [ ]:
pd.DataFrame([
              [word_train, sentence_train],
              [word_test, sentence_test],
              [word_dev, sentence_dev]
], columns = ['Word Accuracy (%)', 'Sentence Accuracy (%)'], index = ["Train", "Test", "Dev"])


,Word Accuracy (%),Sentence Accuracy (%)
Train,93.9776,40.6111
Test,84.5284,19.1011
Dev,85.5558,15.8163


**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using the Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.

In [ ]:
# helper functions
def accuracy(data, tags):
  correct_word_counter = word_count = correct_sentence_counter = 0

  for (sentence, words_tags) in zip(data, tags):
    word_count += len(words_tags)
    sentence_success = True 
    for word in range(len(sentence)):
      tag = words_tags[word][1] if isinstance(words_tags[word], tuple) else words_tags[word]
      if tag == sentence[word][1]:
        correct_word_counter += 1
      else:
        sentence_success = False
    if sentence_success:
      correct_sentence_counter += 1
  
  w = round((correct_word_counter / word_count) * 100, 4)
  s = round((correct_sentence_counter / len(tags)) * 100, 4)
  return w,s

In [ ]:
import random

class hmm_tagger:
  A = B = Pi = None
  words_ids = {}
  pos_ids = {}

  def create_matrices(self, data):
    word_count = 0
    tag_count = 0
    for sentence in data:
      for (word, tag) in sentence:
        if self.words_ids.get(word, -1) == -1:
          self.words_ids[word] = word_count
          word_count += 1
        if self.pos_ids.get(tag, -1) == -1:
          self.pos_ids[tag] = tag_count
          tag_count += 1
    self.A = np.zeros((tag_count, tag_count))
    self.B = np.zeros((tag_count, word_count))
    self.Pi = np.zeros(tag_count)
    
  def train(self, data):
    self.create_matrices(data)

    for sentence in data:
      (word, tag) = sentence[0]
      self.Pi[self.pos_ids[tag]] += 1
      self.B[self.pos_ids[tag]][self.words_ids[word]] += 1
      for i,j in zip(range(len(sentence)), range(1, len(sentence))):
        tag_i = sentence[i][1]
        (word_j, tag_j) = sentence[j]
        self.A[self.pos_ids[tag_i]][self.pos_ids[tag_j]] += 1
        self.B[self.pos_ids[tag_j]][self.words_ids[word_j]] += 1
    
    for i in range(self.Pi.shape[0]):
      self.A[i] /= (self.A[i].sum() or 1)
      self.B[i] /= (self.B[i].sum() or 1)
    self.Pi /= len(data)

  def evaluate(self, data):
    pos_tags = sorted(self.pos_ids, key=self.pos_ids.get)
    tags = []

    for sentence in data:
      observations = [self.words_ids.get(word, -1) for (word, tag) in sentence]
      sentence_tags = []
      last = 0
      for obs in range(len(observations)):
        if observations[obs] == -1:
          if last != obs:
            sentence_tags += viterbi(observations[last:obs], self.A, self.B, self.Pi)
          sentence_tags.append(random.choice(list(self.pos_ids.values())))
          last = obs+1
      if last < len(observations):
        sentence_tags += viterbi(observations[last:], self.A, self.B, self.Pi)
      tags.append([pos_tags[tag] for tag in sentence_tags])

    return accuracy(data, tags)

In [ ]:
# Viterbi
def viterbi (observations, A, B, Pi):
  N = A.shape[0]
  T = len(observations)
  delta = np.zeros((N, T))
  psi = np.zeros((N, T))
  best_sequence = np.zeros(T, dtype=int)
  for n in range(N):
    delta[n][0] = B[n][observations[0]] * Pi[n]
    psi[n][0] = 0
  for t in range(1,T):
    for n in range(N):
      (max_val, j) = max([(delta[j][t-1] * A[j][n], j) for j in range(N)], key= lambda p: p[0])
      delta[n][t] = B[n][observations[t]] * max_val
      psi[n][t] = j

  best_sequence[T-1] = max([(delta[j][T-1], j) for j in range(N)], key= lambda p: p[0])[1]
  for t in reversed(range(T-1)):
    best_sequence[t] = psi[best_sequence[t+1]][t+1]

  return list(best_sequence)

**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over both, the dev and test datasets. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
from nltk.tag import tnt 

%cd /content/UD_English-GUM/
!git checkout 2c8b062269f2d2d3d62405c82d8c25cf24f705dd
%cd ..
def read_data(file_name):
  data = []
  file_path = f'UD_English-GUM/en_gum-ud-{file_name}.conllu'
  for sentence in conllutils.read_conllu(file_path):
    data.append([(token['form'], token['upos']) for token in sentence.tokens()])
  return data

train_data = read_data('train')
test_data = read_data('test')
dev_data = read_data('dev')

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)
print(tnt_pos_tagger.evaluate(test_data))

/content/UD_English-GUM
HEAD is now at 2c8b062 Updated statistics.
/content
0.802335803089288


Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
import pandas as pd
from nltk.tag.util import untag

tnt_dev_tagged_sents = tnt_pos_tagger.tag_sents(untag(sent) for sent in dev_data)
tnt_test_tagged_sents = tnt_pos_tagger.tag_sents(untag(sent) for sent in test_data)
tnt_dev_results = accuracy(dev_data, tnt_dev_tagged_sents)
tnt_test_results = accuracy(test_data, tnt_test_tagged_sents)

simple_pos_tagger = simple_tagger()
simple_pos_tagger.train(train_df)
simple_dev_results = simple_pos_tagger.evaluate(dev_df)
simple_test_results =  simple_pos_tagger.evaluate(test_df)

train_data = read_data('train')
hmm_pos_tagger = hmm_tagger()
hmm_pos_tagger.train(train_data)
hmm_dev_results = hmm_pos_tagger.evaluate(dev_data)
hmm_test_results =  hmm_pos_tagger.evaluate(test_data)

In [ ]:
pd.DataFrame([
                [simple_dev_results[0], simple_dev_results[1], simple_test_results[0], simple_test_results[1]], 
                [hmm_dev_results[0], hmm_dev_results[1], hmm_test_results[0], hmm_test_results[1]],
                [tnt_dev_results[0], tnt_dev_results[1], tnt_test_results[0], tnt_test_results[1]],
], columns=['Word Accuracy (Dev)', 'Sentence Accuracy (Dev)', 'Word Accuracy (Test)', 'Sentence Accuracy (Test)'], index=['Simple', 'HMM', 'MEMM'])

,Word Accuracy (Dev),Sentence Accuracy (Dev),Word Accuracy (Test),Sentence Accuracy (Test)
Simple,85.5558,15.8163,84.5284,19.1011
HMM,83.2222,13.1378,80.9243,12.6966
MEMM,82.7863,12.7551,80.2336,12.3596
